# 测试集图像分类预测结果

使用训练好的图像分类模型，预测测试集的所有图像，得到预测结果表格。

同济子豪兄：https://space.bilibili.com/1900783

[代码运行云GPU环境](https://featurize.cn/?s=d7ce99f842414bfcaea5662a97581bd1)：GPU RTX 3060、CUDA v11.2

## 导入工具包

In [1]:
import os
from tqdm import tqdm

import numpy as np
import pandas as pd

from PIL import Image

import torch
import torch.nn.functional as F

# 有 GPU 就用 GPU，没有就用 CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cpu


## 图像预处理

In [2]:
from torchvision import transforms

# # 训练集图像预处理：缩放裁剪、图像增强、转 Tensor、归一化
# train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
#                                       transforms.RandomHorizontalFlip(),
#                                       transforms.ToTensor(),
#                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#                                      ])

# 测试集图像预处理-RCTN：缩放、裁剪、转 Tensor、归一化
test_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         mean=[0.485, 0.456, 0.406], 
                                         std=[0.229, 0.224, 0.225])
                                    ])

## 载入测试集（和训练代码教程相同）

In [3]:
# 数据集文件夹路径
dataset_dir = os.path.join('..', '..', 'data', 'fruit30_split')
test_path = os.path.join(dataset_dir, 'val')
from torchvision import datasets
# 载入测试集
test_dataset = datasets.ImageFolder(test_path, test_transform)
print('测试集图像数量', len(test_dataset))
print('类别个数', len(test_dataset.classes))
print('各类别名称', test_dataset.classes)
# 载入类别名称 和 ID索引号 的映射字典
idx_to_labels = np.load('idx_to_labels.npy', allow_pickle=True).item()
# 获得类别名称
classes = list(idx_to_labels.values())
print(classes)

测试集图像数量 1078
类别个数 30
各类别名称 ['哈密瓜', '圣女果', '山竹', '杨梅', '柚子', '柠檬', '桂圆', '梨', '椰子', '榴莲', '火龙果', '猕猴桃', '石榴', '砂糖橘', '胡萝卜', '脐橙', '芒果', '苦瓜', '苹果-红', '苹果-青', '草莓', '荔枝', '菠萝', '葡萄-白', '葡萄-红', '西瓜', '西红柿', '车厘子', '香蕉', '黄瓜']
['哈密瓜', '圣女果', '山竹', '杨梅', '柚子', '柠檬', '桂圆', '梨', '椰子', '榴莲', '火龙果', '猕猴桃', '石榴', '砂糖橘', '胡萝卜', '脐橙', '芒果', '苦瓜', '苹果-红', '苹果-青', '草莓', '荔枝', '菠萝', '葡萄-白', '葡萄-红', '西瓜', '西红柿', '车厘子', '香蕉', '黄瓜']


## 导入训练好的模型

In [5]:
model = torch.load('../../checkpoint/fruit30_pytorch_C1.pth', map_location=torch.device('cpu'))
model = model.eval().to(device)

## 表格A-测试集图像路径及标注

In [6]:
test_dataset.imgs[:10]

[('..\\..\\data\\fruit30_split\\val\\哈密瓜\\106.jpg', 0),
 ('..\\..\\data\\fruit30_split\\val\\哈密瓜\\109.jpg', 0),
 ('..\\..\\data\\fruit30_split\\val\\哈密瓜\\114.jpg', 0),
 ('..\\..\\data\\fruit30_split\\val\\哈密瓜\\116.jpg', 0),
 ('..\\..\\data\\fruit30_split\\val\\哈密瓜\\118.png', 0),
 ('..\\..\\data\\fruit30_split\\val\\哈密瓜\\123.jpg', 0),
 ('..\\..\\data\\fruit30_split\\val\\哈密瓜\\127.jpg', 0),
 ('..\\..\\data\\fruit30_split\\val\\哈密瓜\\129.jpg', 0),
 ('..\\..\\data\\fruit30_split\\val\\哈密瓜\\131.jpg', 0),
 ('..\\..\\data\\fruit30_split\\val\\哈密瓜\\133.jpg', 0)]

In [7]:
img_paths = [each[0] for each in test_dataset.imgs]

In [8]:
df = pd.DataFrame()
df['图像路径'] = img_paths
df['标注类别ID'] = test_dataset.targets
df['标注类别名称'] = [idx_to_labels[ID] for ID in test_dataset.targets]

In [11]:
df

,图像路径,标注类别ID,标注类别名称
0,fruit30_split/val/哈密瓜/106.jpg,0,哈密瓜
1,fruit30_split/val/哈密瓜/109.jpg,0,哈密瓜
2,fruit30_split/val/哈密瓜/114.jpg,0,哈密瓜
3,fruit30_split/val/哈密瓜/116.jpg,0,哈密瓜
4,fruit30_split/val/哈密瓜/118.png,0,哈密瓜
...,...,...,...
1074,fruit30_split/val/黄瓜/87.jpg,29,黄瓜
1075,fruit30_split/val/黄瓜/9.jpg,29,黄瓜
1076,fruit30_split/val/黄瓜/91.png,29,黄瓜
1077,fruit30_split/val/黄瓜/94.jpg,29,黄瓜


## 表格B-测试集每张图像的图像分类预测结果，以及各类别置信度

In [9]:
# 记录 top-n 预测结果
n = 3

In [10]:
df_pred = pd.DataFrame()
for idx, row in tqdm(df.iterrows()):
    img_path = row['图像路径']
    img_pil = Image.open(img_path).convert('RGB')
    input_img = test_transform(img_pil).unsqueeze(0).to(device) # 预处理
    pred_logits = model(input_img) # 执行前向预测，得到所有类别的 logit 预测分数
    pred_softmax = F.softmax(pred_logits, dim=1) # 对 logit 分数做 softmax 运算

    pred_dict = {}

    top_n = torch.topk(pred_softmax, n) # 取置信度最大的 n 个结果
    pred_ids = top_n[1].cpu().detach().numpy().squeeze() # 解析出类别
    
    # top-n 预测结果
    for i in range(1, n+1):
        pred_dict['top-{}-预测ID'.format(i)] = pred_ids[i-1]
        pred_dict['top-{}-预测名称'.format(i)] = idx_to_labels[pred_ids[i-1]]
    pred_dict['top-n预测正确'] = row['标注类别ID'] in pred_ids
    # 每个类别的预测置信度
    for idx, each in enumerate(classes):
        pred_dict['{}-预测置信度'.format(each)] = pred_softmax[0][idx].cpu().detach().numpy()
        
    df_pred = pd.concat([df_pred, pd.DataFrame([pred_dict])], ignore_index=True)

1078it [01:06, 16.24it/s]


In [12]:
df_pred

,top-1-预测ID,top-1-预测名称,top-2-预测ID,top-2-预测名称,top-3-预测ID,top-3-预测名称,top-n预测正确,哈密瓜-预测置信度,圣女果-预测置信度,山竹-预测置信度,...,草莓-预测置信度,荔枝-预测置信度,菠萝-预测置信度,葡萄-白-预测置信度,葡萄-红-预测置信度,西瓜-预测置信度,西红柿-预测置信度,车厘子-预测置信度,香蕉-预测置信度,黄瓜-预测置信度
0,4,柚子,0,哈密瓜,5,柠檬,True,0.023467984,2.6520058e-05,2.592328e-06,...,3.521001e-07,2.3089494e-06,2.254164e-06,1.5042169e-06,5.554669e-07,0.00013355633,1.1679475e-05,3.9260376e-06,3.7907197e-05,1.8140366e-07
1,0,哈密瓜,6,桂圆,8,椰子,True,0.7288786,1.6811409e-06,1.4247325e-05,...,2.91054e-07,1.3115365e-05,7.0390905e-07,0.0014048791,2.5694319e-05,0.00018166432,2.7659857e-07,1.7792878e-05,0.0010005179,4.129508e-06
2,0,哈密瓜,26,西红柿,2,山竹,True,0.90622133,0.0047145444,0.014580457,...,0.004027732,0.008896847,0.00033528765,0.005858226,0.0019321974,0.00041067533,0.019276153,0.00024098536,0.0036702661,0.00014774811
3,0,哈密瓜,16,芒果,9,榴莲,True,0.9112893,6.284457e-05,1.33306685e-05,...,1.9135297e-05,0.0003439658,1.4993935e-05,0.00012387977,1.9475951e-07,0.00013048199,5.474818e-05,2.5504667e-07,0.009668104,8.4941974e-05
4,4,柚子,11,猕猴桃,0,哈密瓜,True,0.01908352,0.0004770333,8.653565e-05,...,0.0019592957,6.960726e-05,4.3011045e-05,0.01677096,3.2436452e-05,0.0008338203,0.00061958056,1.0788814e-07,0.0013070244,0.00022736086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,29,黄瓜,25,西瓜,17,苦瓜,True,4.7360216e-05,5.4742024e-08,3.2455237e-08,...,1.5143431e-08,3.0627225e-09,8.348586e-10,3.0200645e-06,1.6202571e-10,0.0018049072,1.0073055e-07,8.860658e-09,1.5277918e-06,0.9978435
1074,29,黄瓜,17,苦瓜,19,苹果-青,True,1.0950505e-06,1.4452239e-08,3.084971e-08,...,8.3907004e-08,2.495038e-08,1.8800838e-08,5.5092247e-08,3.6121677e-09,4.165265e-06,2.5414787e-08,1.10794645e-07,7.8369e-07,0.9994579
1075,29,黄瓜,17,苦瓜,23,葡萄-白,True,0.0025625331,0.00029212737,6.7765555e-05,...,0.00026492908,0.0037463426,3.7822247e-05,0.0043213624,5.0860122e-05,2.7666754e-05,4.0237235e-05,3.5511996e-06,0.00026483182,0.8883497
1076,29,黄瓜,17,苦瓜,10,火龙果,True,4.8565767e-06,4.8560836e-05,1.3399575e-06,...,0.0007608221,7.951188e-06,0.00016525609,0.00033323452,9.151688e-05,0.001328466,1.9984273e-05,1.1606063e-05,0.00021495899,0.9693536


## 拼接AB两张表格

In [13]:
df = pd.concat([df, df_pred], axis=1)

In [16]:
df

,图像路径,标注类别ID,标注类别名称,top-1-预测ID,top-1-预测名称,top-2-预测ID,top-2-预测名称,top-3-预测ID,top-3-预测名称,top-n预测正确,...,草莓-预测置信度,荔枝-预测置信度,菠萝-预测置信度,葡萄-白-预测置信度,葡萄-红-预测置信度,西瓜-预测置信度,西红柿-预测置信度,车厘子-预测置信度,香蕉-预测置信度,黄瓜-预测置信度
0,..\..\data\fruit30_split\val\哈密瓜\106.jpg,0,哈密瓜,4,柚子,0,哈密瓜,5,柠檬,True,...,3.521001e-07,2.3089494e-06,2.254164e-06,1.5042169e-06,5.554669e-07,0.00013355633,1.1679475e-05,3.9260376e-06,3.7907197e-05,1.8140366e-07
1,..\..\data\fruit30_split\val\哈密瓜\109.jpg,0,哈密瓜,0,哈密瓜,6,桂圆,8,椰子,True,...,2.91054e-07,1.3115365e-05,7.0390905e-07,0.0014048791,2.5694319e-05,0.00018166432,2.7659857e-07,1.7792878e-05,0.0010005179,4.129508e-06
2,..\..\data\fruit30_split\val\哈密瓜\114.jpg,0,哈密瓜,0,哈密瓜,26,西红柿,2,山竹,True,...,0.004027732,0.008896847,0.00033528765,0.005858226,0.0019321974,0.00041067533,0.019276153,0.00024098536,0.0036702661,0.00014774811
3,..\..\data\fruit30_split\val\哈密瓜\116.jpg,0,哈密瓜,0,哈密瓜,16,芒果,9,榴莲,True,...,1.9135297e-05,0.0003439658,1.4993935e-05,0.00012387977,1.9475951e-07,0.00013048199,5.474818e-05,2.5504667e-07,0.009668104,8.4941974e-05
4,..\..\data\fruit30_split\val\哈密瓜\118.png,0,哈密瓜,4,柚子,11,猕猴桃,0,哈密瓜,True,...,0.0019592957,6.960726e-05,4.3011045e-05,0.01677096,3.2436452e-05,0.0008338203,0.00061958056,1.0788814e-07,0.0013070244,0.00022736086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,..\..\data\fruit30_split\val\黄瓜\87.jpg,29,黄瓜,29,黄瓜,25,西瓜,17,苦瓜,True,...,1.5143431e-08,3.0627225e-09,8.348586e-10,3.0200645e-06,1.6202571e-10,0.0018049072,1.0073055e-07,8.860658e-09,1.5277918e-06,0.9978435
1074,..\..\data\fruit30_split\val\黄瓜\9.jpg,29,黄瓜,29,黄瓜,17,苦瓜,19,苹果-青,True,...,8.3907004e-08,2.495038e-08,1.8800838e-08,5.5092247e-08,3.6121677e-09,4.165265e-06,2.5414787e-08,1.10794645e-07,7.8369e-07,0.9994579
1075,..\..\data\fruit30_split\val\黄瓜\91.png,29,黄瓜,29,黄瓜,17,苦瓜,23,葡萄-白,True,...,0.00026492908,0.0037463426,3.7822247e-05,0.0043213624,5.0860122e-05,2.7666754e-05,4.0237235e-05,3.5511996e-06,0.00026483182,0.8883497
1076,..\..\data\fruit30_split\val\黄瓜\94.jpg,29,黄瓜,29,黄瓜,17,苦瓜,10,火龙果,True,...,0.0007608221,7.951188e-06,0.00016525609,0.00033323452,9.151688e-05,0.001328466,1.9984273e-05,1.1606063e-05,0.00021495899,0.9693536


## 导出完整表格

In [17]:
df.to_csv('测试集预测结果.csv', index=False)